In [99]:
import pandas as pd
from nltk import tokenize
from gensim.summarization import summarize
import re
import rouge
import nltk
import os
import sys

In [100]:
pwd

'C:\\Users\\User\\Desktop'

In [101]:
cd C:\\Users\\User\\Desktop

C:\Users\User\Desktop


In [102]:
example = pd.read_json('example_texts.json')
example

,0
0,Увидеть мысль\n\nЯпонским ученым удалось распо...
1,"Карбофос\n\nКарбофос ( O , O - Диметил - S - (..."
2,Киев идет на уступки : Яценюк передает власть ...
3,« Лужков . Итоги »\nКонец эры хозяйственника и...
4,... в ходе написания ходатайства : сделать его...


In [103]:
sample_text = example[0][0]
sample_text

'Увидеть мысль\n\nЯпонским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг .\n\nКак пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение , увиденное человеком , опираясь только на сканирование электрических сигналов мозга .\n\nВ ходе эксперимента исследователи продемонстрировали испытуемым серию из 440 различных картинок , представлявших собой произвольное сочетание темных и светлых пятен , на 100-пиксельном экране .\nВ процессе демонстрации измерялась активность в зрительной коре головного мозга с помощью функционального магниторезонансного сканера .\nПосле этого испытуемым были показаны буквы , составляющие слово neuron , и элементарные геометрические фигуры .\nНа основе сравнения предыдущей серии изображений и показаний сканера исследователям удалось правильно реконструировать новые изображения по сигналам мозговой ак

In [104]:
DOCUMENT = sample_text
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()
DOCUMENT

'Увидеть мысль Японским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг . Как пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение , увиденное человеком , опираясь только на сканирование электрических сигналов мозга . В ходе эксперимента исследователи продемонстрировали испытуемым серию из 440 различных картинок , представлявших собой произвольное сочетание темных и светлых пятен , на 100-пиксельном экране . В процессе демонстрации измерялась активность в зрительной коре головного мозга с помощью функционального магниторезонансного сканера . После этого испытуемым были показаны буквы , составляющие слово neuron , и элементарные геометрические фигуры . На основе сравнения предыдущей серии изображений и показаний сканера исследователям удалось правильно реконструировать новые изображения по сигналам мозговой активности . З

In [105]:
summarized_text = summarize(DOCUMENT, ratio=0.2, split=False)
print(summarized_text)

Как пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение , увиденное человеком , опираясь только на сканирование электрических сигналов мозга .


In [106]:
dataset = pd.read_json('dataset_43428_1.txt')
dataset.index_col = 0
dataset

,0
0,В Сербии арестован последний военный преступни...
1,АнтиПоттер\nРуководство по эксплуатации волшеб...
2,Авто по карману\nМихаил Прохоров собирается за...
3,В Казахстане состоялись выборы Президента\n\nВ...
4,В Москве подвели итоги Международной конференц...
...,...
195,13-я экспедиция МКС\n\n30 марта 2006 года с ко...
196,Пираты не признали вину\n\nВ американском горо...
197,Аллегория удачи .\nВизит Дмитрия Медведева в К...
198,Тем временем участницы панк-группы встретились...


In [107]:
dataset2 = str(dataset)

In [108]:
dataset2 = re.sub(r'\[[0-9]*\]', ' ', dataset2)
dataset2 = re.sub(r'\s+', ' ', dataset2)

In [109]:
formatted_dataset = re.sub('[^a-zA-Z]', ' ', dataset2 )
formatted_dataset = re.sub(r'\s+', ' ', formatted_dataset)

In [110]:
sentence_list = nltk.sent_tokenize(dataset2)
dataset2

' 0 0 В Сербии арестован последний военный преступни... 1 АнтиПоттер\\nРуководство по эксплуатации волшеб... 2 Авто по карману\\nМихаил Прохоров собирается за... 3 В Казахстане состоялись выборы Президента\\n\\nВ... 4 В Москве подвели итоги Международной конференц... .. ... 195 13-я экспедиция МКС\\n\\n30 марта 2006 года с ко... 196 Пираты не признали вину\\n\\nВ американском горо... 197 Аллегория удачи .\\nВизит Дмитрия Медведева в К... 198 Тем временем участницы панк-группы встретились... 199 Статья 54 .\\nОбщие вопросы исчисления налогово... [200 rows x 1 columns]'

In [111]:
stopwords = nltk.corpus.stopwords.words('russian')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_dataset):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [112]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

In [113]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [114]:
import heapq
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
print(summary)

In [115]:
def simpleword_count(text):
    word_list = tokenize.word_tokenize(text)
    return len(word_list)

def word_count_df(row):
    text = row[0]
    word_list = tokenize.word_tokenize(text)
    return len(word_list)


def clean_text(row):
    text = row[0]
    text = text.lower()
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    return text

def summary_text(row):
    text = row['clean_text']
    sentences = tokenize.sent_tokenize(text)
    wc = simpleword_count(text)
    if len(sentences) == 1:
        return text
    if wc <= 1500:
        summarized_text = summarize(text, ratio=0.2, split=False)
    elif wc >= 1500:
        summarized_text = summarize(text, word_count=300, split=False)
    
    return summarized_text

def summary_word_count_df(row):
    text = row['summary_text']
    word_list = tokenize.word_tokenize(text)
    return len(word_list)

In [116]:
dataset['orig_word_count'] = dataset.apply(lambda row: word_count_df(row), axis=1)
dataset['clean_text'] = dataset.apply(lambda row: clean_text(row), axis=1)
dataset['summary_text'] = dataset.apply(lambda row: summary_text(row), axis=1)
dataset['summary_word_count'] = dataset.apply(lambda row: summary_word_count_df(row), axis=1)
dataset

,0,orig_word_count,clean_text,summary_text,summary_word_count
0,В Сербии арестован последний военный преступни...,213,в сербии арестован последний военный преступни...,"за информацию , способную помочь в поимке гора...",50
1,АнтиПоттер\nРуководство по эксплуатации волшеб...,1918,антипоттер руководство по эксплуатации волшебн...,мне уже говорили мои молодые коллеги и просто ...,296
2,Авто по карману\nМихаил Прохоров собирается за...,110,авто по карману михаил прохоров собирается зан...,интересы у олигарха михаила прохорова самые ра...,18
3,В Казахстане состоялись выборы Президента\n\nВ...,376,в казахстане состоялись выборы президента в во...,по предварительным данным опроса избирателей н...,51
4,В Москве подвели итоги Международной конференц...,880,в москве подвели итоги международной конференц...,в москве подвели итоги международной конференц...,228
...,...,...,...,...,...
195,13-я экспедиция МКС\n\n30 марта 2006 года с ко...,576,13-я экспедиция мкс 30 марта 2006 года с космо...,13-я экспедиция мкс 30 марта 2006 года с космо...,162
196,Пираты не признали вину\n\nВ американском горо...,253,пираты не признали вину в американском городе ...,"кроме того , в руках вмс сша у берегов сомали ...",66
197,Аллегория удачи .\nВизит Дмитрия Медведева в К...,1136,аллегория удачи . визит дмитрия медведева в ки...,"трудно сказать , смирились ли сторонники викто...",344
198,Тем временем участницы панк-группы встретились...,519,тем временем участницы панк-группы встретились...,тем временем участницы панк-группы встретились...,100


In [117]:
dataset.to_json('gensim_output_dataset.json')
result = pd.DataFrame(dataset['summary_text'])
result

,summary_text
0,"за информацию , способную помочь в поимке гора..."
1,мне уже говорили мои молодые коллеги и просто ...
2,интересы у олигарха михаила прохорова самые ра...
3,по предварительным данным опроса избирателей н...
4,в москве подвели итоги международной конференц...
...,...
195,13-я экспедиция мкс 30 марта 2006 года с космо...
196,"кроме того , в руках вмс сша у берегов сомали ..."
197,"трудно сказать , смирились ли сторонники викто..."
198,тем временем участницы панк-группы встретились...


In [118]:
dataset.to_csv('gensim_output_dataset.csv')
data = pd.read_csv('gensim_output_dataset.csv')
data

,Unnamed: 0,0,orig_word_count,clean_text,summary_text,summary_word_count
0,0,В Сербии арестован последний военный преступни...,213,в сербии арестован последний военный преступни...,"за информацию , способную помочь в поимке гора...",50
1,1,АнтиПоттер\nРуководство по эксплуатации волшеб...,1918,антипоттер руководство по эксплуатации волшебн...,мне уже говорили мои молодые коллеги и просто ...,296
2,2,Авто по карману\nМихаил Прохоров собирается за...,110,авто по карману михаил прохоров собирается зан...,интересы у олигарха михаила прохорова самые ра...,18
3,3,В Казахстане состоялись выборы Президента\n\nВ...,376,в казахстане состоялись выборы президента в во...,по предварительным данным опроса избирателей н...,51
4,4,В Москве подвели итоги Международной конференц...,880,в москве подвели итоги международной конференц...,в москве подвели итоги международной конференц...,228
...,...,...,...,...,...,...
195,195,13-я экспедиция МКС\n\n30 марта 2006 года с ко...,576,13-я экспедиция мкс 30 марта 2006 года с космо...,13-я экспедиция мкс 30 марта 2006 года с космо...,162
196,196,Пираты не признали вину\n\nВ американском горо...,253,пираты не признали вину в американском городе ...,"кроме того , в руках вмс сша у берегов сомали ...",66
197,197,Аллегория удачи .\nВизит Дмитрия Медведева в К...,1136,аллегория удачи . визит дмитрия медведева в ки...,"трудно сказать , смирились ли сторонники викто...",344
198,198,Тем временем участницы панк-группы встретились...,519,тем временем участницы панк-группы встретились...,тем временем участницы панк-группы встретились...,100


In [119]:
cwd = os.getcwd()

In [120]:
# initialize setting of ROUGE, eval ROUGE-1, 2, SU4, L
from rouge import Rouge

In [121]:
from rouge import Rouge 

#hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"
#reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students ' knowledge of even ts in the news"

hyp = str(data.orig_word_count)
ref = str(data.summary_word_count)

rouge = Rouge()

In [122]:
scores = rouge.get_scores(hyp, ref)
scores

[{'rouge-1': {'f': 0.5769230719230769,
   'p': 0.5769230769230769,
   'r': 0.5769230769230769},
  'rouge-2': {'f': 0.1199999950000002, 'p': 0.12, 'r': 0.12},
  'rouge-l': {'f': 0.5769230719230769,
   'p': 0.5769230769230769,
   'r': 0.5769230769230769}}]

https://radimrehurek.com/gensim/summarization/summariser.html

https://github.com/pltrdy/rouge
    
https://stackabuse.com/text-summarization-with-nltk-in-python/